In [1]:
# save_reddiffpulsing_frames.py
import numpy as np
import matplotlib.pyplot as plt
import os

# === CONFIGURATION ===
T = 10
h = 0.01
frame_dt = 0.1
periods = 30
omega = (6 * np.pi) / 10
A = 0.1
epsilon = 0.25

# particle dot size (visual only)
blob_radius = 0.0002

# === GRID ===
x_vals = np.linspace(0, 2, 300)
y_vals = np.linspace(0, 1, 150)
X0, Y0 = np.meshgrid(x_vals, y_vals)
dx = x_vals[1] - x_vals[0]
dy = y_vals[1] - y_vals[0]

# === FUNCTIONS ===
def a(t): return epsilon * np.sin(omega * t)
def b(t): return 1 - 2 * epsilon * np.sin(omega * t)
def f(x, t): return a(t) * x**2 + b(t) * x
def df_dx(x, t): return 2 * a(t) * x + b(t)

def compute_flow(X, Y, t):
    fx = f(X, t)
    dfx = df_dx(X, t)
    x_dot = -np.pi * A * np.sin(np.pi * fx) * np.cos(np.pi * Y)
    y_dot = np.pi * A * np.cos(np.pi * fx) * np.sin(np.pi * Y) * dfx
    return x_dot, y_dot

def rk4_flow(X, Y, t, h):
    k1x, k1y = compute_flow(X, Y, t)
    k2x, k2y = compute_flow(X + 0.5*h*k1x, Y + 0.5*h*k1y, t + 0.5*h)
    k3x, k3y = compute_flow(X + 0.5*h*k2x, Y + 0.5*h*k2y, t + 0.5*h)
    k4x, k4y = compute_flow(X + h*k3x, Y + h*k3y, t + h)
    X_new = X + (h/6)*(k1x + 2*k2x + 2*k3x + k4x)
    Y_new = Y + (h/6)*(k1y + 2*k2y + 2*k3y + k4y)
    return X_new, Y_new

def compute_ftle(X0, Y0, t0, T, h):
    X, Y = X0.copy(), Y0.copy()
    for t in np.arange(t0, t0+T+h, h):
        X, Y = rk4_flow(X, Y, t, h)
    J11 = (X[2:,1:-1] - X[:-2,1:-1]) / (2*dx)
    J12 = (X[1:-1,2:] - X[1:-1,:-2]) / (2*dy)
    J21 = (Y[2:,1:-1] - Y[:-2,1:-1]) / (2*dx)
    J22 = (Y[1:-1,2:] - Y[1:-1,:-2]) / (2*dy)
    ftle = np.full(J11.shape, np.nan)
    for i in range(ftle.shape[0]):
        for j in range(ftle.shape[1]):
            J = np.array([[J11[i,j], J12[i,j]], [J21[i,j], J22[i,j]]])
            delta = J.T @ J
            eigvals = np.linalg.eigvalsh(delta)
            max_eig = np.max(eigvals)
            if max_eig > 1e-12 and np.isfinite(max_eig):
                ftle[i,j] = (1.0/T) * np.log(np.sqrt(max_eig))
    ftle_full = np.full_like(X0, np.nan, dtype=float)
    ftle_full[1:-1,1:-1] = ftle
    return ftle_full

# --- NEW: robust ridge-top picker (column-wise maxima on top band) ---
def pick_top_ridge_point(ftle_field, X0, Y0, y_cut=0.94):
    top_rows = np.where(Y0[:,0] >= y_cut)[0]
    if top_rows.size == 0:
        # fallback to global max
        i, j = np.unravel_index(np.nanargmax(ftle_field), ftle_field.shape)
        return X0[i,j], Y0[i,j]
    i_min = top_rows[0]
    best = (-np.inf, None, None)
    # for each column, take the local maximum within the top band
    for j in range(ftle_field.shape[1]):
        col = ftle_field[i_min:, j]
        if not np.isfinite(col).any():
            continue
        i_off = np.nanargmax(col)
        val = col[i_off]
        if np.isfinite(val) and val > best[0]:
            best = (val, i_min + i_off, j)
    if best[1] is None:
        i, j = np.unravel_index(np.nanargmax(ftle_field), ftle_field.shape)
    else:
        _, i, j = best
    return X0[i,j], Y0[i,j]

# === OUTPUT FOLDER ===
frame_folder = "reddiffpulsing"
os.makedirs(frame_folder, exist_ok=True)

# === (1) Detect the top red ridge once, to place the pulsing start ===
ftle0 = compute_ftle(X0, Y0, t0=0.0, T=T, h=h)
manual_start_x, manual_start_y = pick_top_ridge_point(ftle0, X0, Y0, y_cut=0.94)

# === (2) Pulsing particle arrays ===
particle_X = np.array([])
particle_Y = np.array([])

# === (3) Frame times over 'periods' full drive cycles (period = 10/3) ===
t0_values = np.arange(0, periods*(10/3), frame_dt)

# === MAIN LOOP ===
for frame_idx, t0 in enumerate(t0_values):
    ftle_frame = compute_ftle(X0, Y0, t0, T, h)

    # Drop a new particle at the ridge start (pulsing trail)
    if particle_X.size == 0:
        particle_X = np.array([manual_start_x])
        particle_Y = np.array([manual_start_y])
    else:
        particle_X = np.concatenate([particle_X, [manual_start_x]])
        particle_Y = np.concatenate([particle_Y, [manual_start_y]])

    # Advect ALL particles for this frame interval
    for t in np.arange(t0, t0 + frame_dt + h, h):
        particle_X, particle_Y = rk4_flow(particle_X, particle_Y, t, h)

    # Plot FTLE + pulsing particles
    plt.figure(figsize=(12, 6))
    plt.contourf(X0, Y0, np.ma.masked_invalid(ftle_frame), levels=100, cmap='jet')
    plt.colorbar(label="FTLE")
    plt.scatter(particle_X, particle_Y, color='black', s=(blob_radius*50000), marker='o')
    plt.title(f"Red Ridge Pulsing (top) | t0={t0:.2f}  start=({manual_start_x:.3f},{manual_start_y:.3f})")
    plt.xlabel("X"); plt.ylabel("Y")
    plt.xlim(0, 2); plt.ylim(0, 1); plt.gca().set_aspect('equal')

    filename = os.path.join(frame_folder, f"reddiffpulsing_{frame_idx:04d}.png")
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved frame: {filename}")


Saved frame: reddiffpulsing/reddiffpulsing_0000.png
Saved frame: reddiffpulsing/reddiffpulsing_0001.png
Saved frame: reddiffpulsing/reddiffpulsing_0002.png
Saved frame: reddiffpulsing/reddiffpulsing_0003.png
Saved frame: reddiffpulsing/reddiffpulsing_0004.png
Saved frame: reddiffpulsing/reddiffpulsing_0005.png
Saved frame: reddiffpulsing/reddiffpulsing_0006.png
Saved frame: reddiffpulsing/reddiffpulsing_0007.png
Saved frame: reddiffpulsing/reddiffpulsing_0008.png
Saved frame: reddiffpulsing/reddiffpulsing_0009.png
Saved frame: reddiffpulsing/reddiffpulsing_0010.png
Saved frame: reddiffpulsing/reddiffpulsing_0011.png
Saved frame: reddiffpulsing/reddiffpulsing_0012.png
Saved frame: reddiffpulsing/reddiffpulsing_0013.png
Saved frame: reddiffpulsing/reddiffpulsing_0014.png
Saved frame: reddiffpulsing/reddiffpulsing_0015.png
Saved frame: reddiffpulsing/reddiffpulsing_0016.png
Saved frame: reddiffpulsing/reddiffpulsing_0017.png
Saved frame: reddiffpulsing/reddiffpulsing_0018.png
Saved frame:

In [2]:
# make_reddiffpulsing_movie.py
import cv2
import os

frame_folder = "reddiffpulsing"
output_video = "reddiffpulsing.mp4"
fps = 20

frames = sorted([f for f in os.listdir(frame_folder) if f.endswith(".png")])
if not frames:
    raise ValueError(f"No frames found in folder: {frame_folder}")

first_frame = cv2.imread(os.path.join(frame_folder, frames[0]))
height, width, _ = first_frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

for f in frames:
    img = cv2.imread(os.path.join(frame_folder, f))
    video.write(img)

video.release()
print(f"Movie saved: {output_video}")


[ WARN:0@0.079] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.192] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.253] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.270] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.289] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.409] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.527] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.582] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.693] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.752] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.769] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.787] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
[ WARN:0@0.910] global cap_ffmpeg.cpp:198 write FFmpeg: Failed t

Movie saved: reddiffpulsing.mp4


[ WARN:0@11.094] global cap_ffmpeg.cpp:198 write FFmpeg: Failed to write frame
